<a href="https://colab.research.google.com/github/kevnantony/TSPrediction/blob/main/prophet_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pystan==2.19.1.1 --quiet
!pip install prophet --quiet
!pip install yfinance --quiet

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      Cython>=0.22 and NumPy are required.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pystan
ERROR: Could not build wheels for pystan, which is required to install pyproject.toml-based projects

[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
!pip install localpip
!localpip install prophet


[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Traceback (most recent call last):
  File "/Users/user1/Library/Python/3.11/bin/localpip", line 5, in <module>
    from localpip.__main__ import main
  File "/Users/user1/Library/Python/3.11/lib/python/site-packages/localpip/__main__.py", line 5, in <module>
    from ctypes import Structure, byref, windll
ImportError: cannot import name 'windll' from 'ctypes' (/usr/local/Cellar/python@3.11/3.11.7/Frameworks/Python.framework/Versions/3.11/lib/python3.11/ctypes/__init__.py)


In [3]:
!pip install plotly



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [4]:
today = '2024-01-31'
start_date = '2016-01-01'

eth_df = yf.download('ETH-USD',start_date, today)

eth_df.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-26,"$2,217.44","$2,280.38","$2,196.14","$2,267.20","$2,267.20",9975117607
2024-01-27,"$2,267.32","$2,282.54","$2,252.39","$2,267.89","$2,267.89",5144367230
2024-01-28,"$2,268.19","$2,306.90","$2,242.68","$2,257.21","$2,257.21",7296214994
2024-01-29,"$2,257.00","$2,320.03","$2,237.71","$2,317.06","$2,317.06",8948195551
2024-01-30,"$2,317.44","$2,388.87","$2,298.28","$2,344.49","$2,344.49",10173440062


In [5]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2274 entries, 2017-11-09 to 2024-01-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2274 non-null   float64
 1   High       2274 non-null   float64
 2   Low        2274 non-null   float64
 3   Close      2274 non-null   float64
 4   Adj Close  2274 non-null   float64
 5   Volume     2274 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 124.4 KB


In [6]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [7]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [8]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [9]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds",
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [10]:
df.tail()

,ds,y
2269,2024-01-26,"$2,217.44"
2270,2024-01-27,"$2,267.32"
2271,2024-01-28,"$2,268.19"
2272,2024-01-29,"$2,257.00"
2273,2024-01-30,"$2,317.44"


In [11]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [12]:
m = Prophet(
    seasonality_mode="multiplicative"
)

m.fit(df)

10:58:31 - cmdstanpy - INFO - Chain [1] start processing
10:58:31 - cmdstanpy - INFO - Chain [1] done processing


In [13]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2634,2025-01-25
2635,2025-01-26
2636,2025-01-27
2637,2025-01-28
2638,2025-01-29


In [14]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2634,2025-01-25,"$2,730.27","$1,333.25","$4,107.56"
2635,2025-01-26,"$2,733.27","$1,357.32","$4,197.38"
2636,2025-01-27,"$2,736.04","$1,344.96","$4,145.35"
2637,2025-01-28,"$2,733.61","$1,328.89","$4,161.09"
2638,2025-01-29,"$2,739.58","$1,329.91","$4,130.82"


In [15]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

2225.294206123352

In [26]:
plot_plotly(m, forecast)

In [27]:
plot_components_plotly(m, forecast)